In [1]:
from vmtk import vmtkscripts
import vtk
import vtk.numpy_interface.dataset_adapter as dsa
import numpy as np
import os
import sys


In [2]:
reader = vmtkscripts.vmtkSurfaceReader()
reader.InputFileName = os.path.join(os.getcwd(), 'testsurf.stl')
reader.Execute()
surf = reader.Surface

Reading STL surface file.


In [3]:
import vmtksurfaceconnectivityselector as surfselector

In [4]:
sel = surfselector.vmtkSurfaceConnectivitySelector()
sel.Surface = surf
sel.Execute()

Please position the mouse and press space to add target points, 'u' to undo
Quit renderer


In [ ]:
sel.

In [3]:
connectiv = vtk.vtkPolyDataConnectivityFilter()
connectiv.SetInputData(surf)
connectiv.SetExtractionModeToAllRegions()
connectiv.ColorRegionsOn()
connectiv.Update()

splitSurface = connectiv.GetOutput()

# connectiv.SetExtractionModeToSpecifiedRegions()
# connectiv.AddSpecifiedRegion(3)
# connectiv.AddSpecifiedRegion(1)

# connectiv.Update()

# splitSurface = connectiv.GetOutput()

In [16]:
arr = splitSurface.GetPointData().GetArray('RegionId')

In [23]:
arr.GetValue(9)

0

In [29]:
splitSurface.GetPoint(0)

(361.91217041015625, 247.83731079101562, 201.5906219482422)

In [30]:
splitSurface.FindPoint(361.91217041015625, 247.83731079101562, 201.5906219482422)

0

In [24]:
connectivityFilter = vtk.vtkPolyDataConnectivityFilter()
connectivityFilter.SetInputData(splitSurface)
connectivityFilter.ScalarConnectivityOn()
scalarRange = [1, 1]
connectivityFilter.SetScalarRange(scalarRange)

In [25]:
connectivityFilter.Update()

In [5]:
viewer = vmtkscripts.vmtkSurfaceViewer()
viewer.Surface = sel.Surface
viewer.Execute()

Quit renderer


In [19]:
dir(arr)

['AbstractArray',
 'AddObserver',
 'Allocate',
 'AoSDataArrayTemplate',
 'BreakOnError',
 'COMPONENT_RANGE',
 'ClearLookup',
 'CopyComponent',
 'CopyComponentNames',
 'CopyInformation',
 'CreateArray',
 'CreateDataArray',
 'CreateDefaultLookupTable',
 'DISCRETE_VALUES',
 'DISCRETE_VALUE_SAMPLE_PARAMETERS',
 'DataArray',
 'DataArrayTemplate',
 'DataChanged',
 'DebugOff',
 'DebugOn',
 'DeepCopy',
 'DeleteMethod',
 'ExportToVoidPointer',
 'FastDelete',
 'FastDownCast',
 'Fill',
 'FillComponent',
 'GUI_HIDE',
 'GetActualMemorySize',
 'GetAddressAsString',
 'GetArrayType',
 'GetClassName',
 'GetCommand',
 'GetComponent',
 'GetComponentName',
 'GetData',
 'GetDataSize',
 'GetDataType',
 'GetDataTypeAsString',
 'GetDataTypeMax',
 'GetDataTypeMin',
 'GetDataTypeRange',
 'GetDataTypeSize',
 'GetDataTypeValueMax',
 'GetDataTypeValueMin',
 'GetDebug',
 'GetElementComponentSize',
 'GetFiniteRange',
 'GetGlobalWarningDisplay',
 'GetInformation',
 'GetLookupTable',
 'GetMTime',
 'GetMaxDiscreteValue

In [7]:
splitSurface.GetPointId(a0)

AttributeError: 'vtkCommonDataModelPython.vtkPolyData' object has no attribute 'GetPointId'

In [6]:
a = splitSurface.GetPoints()

In [7]:
arn = dsa.WrapDataObject(splitSurface)

In [10]:
pd = arn.GetPointData()

In [24]:
splitSurface.GetPointData().

(vtkCommonDataModelPython.vtkPointData)0x11a58a948

In [ ]:
pd = arn.PointData

In [ ]:
pd.GetArray('RegionId')

In [ ]:
v = arn.VTKObject

In [ ]:
v.Get

In [ ]:
arr.GetValue(19000)

In [ ]:
print(splitSurface)

In [ ]:
convert = vmtkscripts.vmtkSurfaceToNumpy()
convert.Surface = splitSurface
convert.Execute()

In [ ]:
ad = convert.ArrayDict

In [ ]:
ad

In [ ]:
viewer = vmtkscripts.vmtkSurfaceViewer()
viewer.Surface = splitSurface
viewer.ArrayName = 'RegionId'
viewer.Legend = 1
viewer.ColorMap = 'rainbow'
viewer.Execute()

In [ ]:
%matplotlib

In [ ]:
print(new)

lca_interactiveselector.py

In [ ]:
from __future__ import absolute_import #NEEDS TO STAY AS TOP LEVEL MODULE FOR Py2-3 COMPATIBILITY
import vtk
import sys
import numpy as np
import copy

from vmtk import vtkvmtk
from vmtk import vmtkrenderer
from vmtk import pypes


class vmtkSeedSelector(object):
    def __init__(self):
        self._Surface = None
        self._SeedIds = None
        self._TargetSeedIds = vtk.vtkFloatArray()
        self.PrintError = None
        self.PrintLog = None
        self.InputText = None
        self.OutputText = None
        self.InputInfo = None
        self.PickedScalarData = None
        self._OutputSeedIds = []

    def SetSurface(self, surface):
        self._Surface = surface

    def GetSurface(self):
        return self._Surface

    def GetTargetSeedIds(self):
        return self._TargetSeedIds
    
    def GetOutputSeedIds(self):
        return self._OutputSeedIds

    def Execute(self):
        pass

class vmtkPickPointSeedSelector(vmtkSeedSelector):

    def __init__(self):
        vmtkSeedSelector.__init__(self)
        self.PickedSeedIds = vtk.vtkIdList()
        self.PickedSeeds = vtk.vtkPolyData()
        self.vmtkRenderer = None
        self.OwnRenderer = 0
        self.Script = None
        

    def UndoCallback(self, obj):
        self.InitializeSeeds()
        self.PickedSeeds.Modified()
        self.vmtkRenderer.RenderWindow.Render()
        self._OutputSeedIds.pop()
        self.PickedActor.GetProperty().SetColor(np.random.random(1), 
                                                np.random.random(1),
                                                np.random.random(1))
        self.PickedActor.GetProperty().SetDiffuse(1.0)
        self.PickedActor.GetProperty().SetSpecular(0.0)
        
        return

    def PickCallback(self, obj):
        picker = vtk.vtkPropPicker()
        eventPosition = self.vmtkRenderer.RenderWindowInteractor.GetEventPosition()
        result = picker.Pick(float(eventPosition[0]),float(eventPosition[1]),0.0,self.vmtkRenderer.Renderer)
        if result is None:
            return
        elif result == 0:
            return
        self.PickedActor = picker.GetActor()
        
        self.PickedActor.GetProperty().SetColor(1.0, 0.0, 0.0)
        self.PickedActor.GetProperty().SetDiffuse(1.0)
        self.PickedActor.GetProperty().SetSpecular(0.0)
        
        self.PickedPoly = self.PickedActor.GetMapper().GetInputAsDataSet()
        self.PickedScalarData = self.PickedPoly.GetPointData().GetScalars()
        
        self._OutputSeedIds.append(copy.deepcopy(self.PickedScalarData.GetValue(0)))
        
        return

        
    def InitializeSeeds(self):
        self.PickedSeedIds.Initialize()
        self.PickedSeeds.Initialize()
        seedPoints = vtk.vtkPoints()
        self.PickedSeeds.SetPoints(seedPoints)

    def Execute(self):

        if (self._Surface == None):
            self.PrintError('vmtkPickPointSeedSelector Error: Surface not set.')
            return

        self._TargetSeedIds.Initialize()

        if not self.vmtkRenderer:
            self.vmtkRenderer = vmtkrenderer.vmtkRenderer()
            self.vmtkRenderer.Initialize()
            self.OwnRenderer = 1

        self.vmtkRenderer.RegisterScript(self.Script)

        ##self.vmtkRenderer.RenderWindowInteractor.AddObserver("KeyPressEvent", self.KeyPressed)
        self.vmtkRenderer.AddKeyBinding('u','Undo.',self.UndoCallback)
        self.vmtkRenderer.AddKeyBinding('space','Add points.',self.PickCallback)

        self.InputInfo('Please position the mouse and press space to add target points, \'u\' to undo\n')

        any = 0
        while any == 0:
            self.InitializeSeeds()
            self.vmtkRenderer.Render()
            try:
                any = self.PickedScalarData.GetValue(0)
            except AttributeError:
                any = None
        if self.OwnRenderer:
            self.vmtkRenderer.Deallocate()


class vmtkInteractiveSelector(pypes.pypeScript):
    def __init__(self):

        pypes.pypeScript.__init__(self)

        self.SeedSelector = None
        self.Surface = None
        self.vmtkRenderer = None
        self.OwnRenderer = 0
        self.OutputSeedIds = []

        self.SetScriptName('vmtkInteractiveSelector')
        self.SetScriptDoc('')
        self.SetInputMembers([
            ['Surface', 'i', 'vtkPolyData', 1, '', 'the input surface', 'vmtksurfacereader'],
            ['vmtkRenderer', 'renderer', 'vmtkRenderer', 1, '', 'external renderer']])
        self.SetOutputMembers([
            ['OutputSeedIds','o','vtkFloatArray',1,'','the output seed ids', '']])

    def Execute(self):

        if self.Surface == None:
            self.PrintError('Error: No input surface.')

        if self.vmtkRenderer is None:
            self.vmtkRenderer = vmtkrenderer.vmtkRenderer()
            self.vmtkRenderer.Initialize()
            self.vmtkRenderer.Renderer.SetBackground(.1, .1, .2 )
            self.OwnRenderer = 1
            
        if self.SeedSelector:
            pass
        else:
            self.SeedSelector = vmtkPickPointSeedSelector()
            self.SeedSelector.vmtkRenderer = self.vmtkRenderer
            self.SeedSelector.Script = self
        
        connectiv = vtk.vtkPolyDataConnectivityFilter()
        connectiv.SetInputData(surf)
        connectiv.SetExtractionModeToAllRegions()
        connectiv.ColorRegionsOn()
        connectiv.Update()
        numRegions = connectiv.GetNumberOfExtractedRegions()
        splitSurface = connectiv.GetOutput()

        for surfaceId in range(numRegions):
            connectiv2 = vtk.vtkPolyDataConnectivityFilter()
            connectiv2.SetInputData = splitSurface
            connectiv2.SetExtractionModeToSpecifiedRegions()
            connectiv2.AddSpecifiedRegion(surfaceId)
            connectiv2.Update()
            surf = connectiv2.GetOutput()
            
            mapper = vtk.vtkPolyDataMapper()
            mapper.SetInputData(surf)
            mapper.ScalarVisibilityOff()
            actor = vtk.vtkActor()
            actor.SetMapper(mapper)

            r = np.random.randint(20, high=70) / 100.0
            g = np.random.randint(20, high=70) / 100.0
            b = np.random.randint(20, high=70) / 100.0
            actor.GetProperty().SetColor(r, g, b)
            actor.GetProperty().SetDiffuse(1.0)
            actor.GetProperty().SetSpecular(0.0)
            actor.GetProperty().SetRepresentationToWireframe()
            actor.GetProperty().EdgeVisibilityOff()

            self.vmtkRenderer.Renderer.AddActor(actor)

        if self.SeedSelector:
            self.SeedSelector.SetSurface(self.Surface)
            self.SeedSelector.InputInfo = self.InputInfo
            self.SeedSelector.InputText = self.InputText
            self.SeedSelector.OutputText = self.OutputText
            self.SeedSelector.PrintError = self.PrintError
            self.SeedSelector.PrintLog = self.PrintLog
            self.SeedSelector.Execute()

            self.OutputSeedIds.append(self.SeedSelector.GetOutputSeedIds())

        if self.OwnRenderer:
            self.vmtkRenderer.Deallocate()


if __name__ == '__main__':
    main = pypes.pypeMain()
    main.Arguments = sys.argv
    main.Execute()

lca_doselection.py

In [ ]:
import numpy as np
import numpy.ma as ma
from vmtk import vmtkscripts
import os
import h5py
import copy
from collections import ChainMap
import vtk
import gc

import lca_clustering_utilities as clustering_utilities
import lca_interactiveselector as interactiveselector
import lca_surface as surface

import dask
from dask.diagnostics import ProgressBar
import dask.distributed
import dask.multiprocessing
from dask import delayed, cache

# convenience class for nested disctionaries
class Vividict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value

def _get_surface_array_dict(volumeDict, labelId, vtkImage):
    '''takes an image, extracts the desired label Id, performs a marching cubes
    surface extraction, labels the surface with the scalar labelId value, and
    returns the surface. 
    ***inputs***
    volumeDict = the volume dictionary in vmtk array dict format
    labelID = the desired label for the mc algorithm to extract. 
    ***output***
    surf = the vtk surface object. 
    
    '''
    
    thresh = vtk.vtkImageThreshold()

    thresh.SetInputData(vtkImage)
    thresh.ThresholdBetween(float(labelId)-0.5, float(labelId)+0.5)
    thresh.SetOutValue(0)
    thresh.Update()
    threshImage = thresh.GetOutput()

    mc = vmtkscripts.vmtkMarchingCubes()
    mc.Image = threshImage
    mc.Level = float(labelId)
    mc.Execute()

    surf = mc.Surface
    scalarLabels = vtk.vtkFloatArray()
    scalarLabels.SetNumberOfValues(1)
    scalarLabels.SetName('LabelIds')
    scalarLabels.SetValue(0, float(labelId))

    surf.GetPointData().SetActiveScalars('LabelIds')
    surf.GetPointData().SetScalars(scalarLabels)

    return surf


def do_selection(volumeDict):
    '''allows the user to interactivly select surfaces extracted from a labeled
    numpy array of non-connected components. returns an vmtk image array dict
    with the selected components included. 
    
    '''

    converter = vmtkscripts.vmtkNumpyToImage()
    converter.ArrayDict = volumeDict
    converter.Execute()
    
    volumeLabels = np.unique(volumeDict['PointData']['ImageScalars'])[1:]
    s_list = [dask.delayed(_get_surface_array_dict)(volumeDict, labelId, converter.Image) for labelId in volumeLabels]
    
    s_list = dask.compute(s_list)[0]
    
    ints = interactiveselector.vmtkInteractiveSelector()
    ints.Surface = s_list
    ints.Execute()
    
    desired_labels = ints.OutputSeedIds[0]

    def _get_mask_array(npImage, labelId):
        mask_labels = ma.masked_values(npImage, labelId)
        return mask_labels.mask

    mask_array = np.zeros_like(volumeDict['PointData']['ImageScalars'])
    for labelId in desired_labels:
        labelId_mask_array = _get_mask_array(volumeDict['PointData']['ImageScalars'], labelId)
        mask_array = np.logical_or(mask_array, labelId_mask_array)

    mask_array = mask_array.astype(np.int16)
    volumeDict['PointData']['ImageScalars'] = volumeDict['PointData']['ImageScalars'] * mask_array
    gc.collect()
    return volumeDict